# PredPatt Tutorial
This is a self-contained tutorial for PredPatt. You'll know how to use PredPatt in your code after going through this short tutorial.

Let's import PredPatt first.

In [1]:
from predpatt import PredPatt

## Input format
PredPatt is built atop [Universal Dependency](http://universaldependencies.org) (UD). Currently, PredPatt takes two kinds of input formats: CoNLL and [Concrete](https://pypi.python.org/pypi/concrete). 

### 1. Load from CoNLL format
The simplest format is CoNLL. Here's an example of input in CoNLL format:

In [32]:
conll_example = """
1	You	you	PRON	PRP	Case=Nom|Person=2|PronType=Prs	2	nsubj	_	_
2	wonder	wonder	VERB	VBP	Mood=Ind|Tense=Pres|VerbForm=Fin	0	root	_	_
3	if	if	SCONJ	IN	_	6	mark	_	_
4	he	he	PRON	PRP	Case=Nom|Gender=Masc|Number=Sing|Person=3|PronType=Prs	6	nsubj	_	_
5	was	be	AUX	VBD	Mood=Ind|Number=Sing|Person=3|Tense=Past|VerbForm=Fin	6	aux	_	_
6	manipulating	manipulate	VERB	VBG	Tense=Pres|VerbForm=Part	2	advcl	_	_
7	the	the	DET	DT	Definite=Def|PronType=Art	8	det	_	_
8	market	market	NOUN	NN	Number=Sing	6	dobj	_	_
9	with	with	ADP	IN	_	12	case	_	_
10	his	he	PRON	PRP$	Gender=Masc|Number=Sing|Person=3|Poss=Yes|PronType=Prs	12	nmod:poss	_	_
11	bombing	bombing	NOUN	NN	Number=Sing	12	compound	_	_
12	targets	target	NOUN	NNS	Number=Plur	6	nmod	_	SpaceAfter=No
13	.	.	PUNCT	.	_	2	punct	_	_
"""

Next, let's load this example using utils provided by PredPatt.

In [3]:
from predpatt import load_conllu

"`load_conllu`" will create a generator to load multiple CoNLL examples (each of them is separated by an empty line). 
Here we just keep the first example:

In [33]:
ud_example = [ud_parse for sent_id, ud_parse in load_conllu(conll_example)][0]

Let's see what this loaded example looks like:

In [20]:
print ud_example.pprint(K=3)  # K is the number of columns

name(Al/0, Zaman/2)           punct(-/1, Zaman/2)         root(Zaman/2, ROOT/-1)
punct(:/3, Zaman/2)           amod(American/4, forces/5)  nsubj(forces/5, killed/6)
parataxis(killed/6, Zaman/2)  name(Shaikh/7, Ani/11)      name(Abdullah/8, Ani/11)
name(al/9, Ani/11)            punct(-/10, Ani/11)         dobj(Ani/11, killed/6)
punct(,/12, Ani/11)           det(the/13, preacher/14)    appos(preacher/14, Ani/11)
case(at/15, mosque/17)        det(the/16, mosque/17)      nmod(mosque/17, killed/6)
case(in/18, town/20)          det(the/19, town/20)        nmod(town/20, mosque/17)
case(of/21, Qaim/22)          nmod(Qaim/22, town/20)      punct(,/23, town/20)
case(near/24, border/27)      det(the/25, border/27)      amod(Syrian/26, border/27)
nmod(border/27, town/20)      punct(./28, Zaman/2)


### 2. Load from Concrete format
Loading from data in Concrete format is also handy. Take the concrete file in the test directory as an example.

In [6]:
from predpatt import load_comm
fpath = "./test/data.100.fine.all.ud.comm"
ud_example = [ud_parse for sent_id, ud_parse in load_comm(fpath)][0]

## Play with PredPatt
With the loaded example, we can easily get a PredPatt object as below:

In [34]:
ppatt = PredPatt(ud_example)

PredPatt provides a straightforward way to show the predicate-argument content. You can enable/disable "`color`" and "`track_rule`" to generate the output that suits your needs.

In [35]:
print " ".join([token.text for token in ppatt.tokens])
print ppatt.pprint(color=True, track_rule=False)

You wonder if he was manipulating the market with his bombing targets .
	?a wonder	[wonder-root]
		?a: You	[You-nsubj]
	?a was manipulating ?b with ?c	[manipulating-advcl]
		?a: he	[he-nsubj]
		?b: the market	[market-dobj]
		?c: his bombing targets	[targets-nmod]


In practice, iterating the extracted predicates is quite easy. Let's iteratively print out the predicate and its arguments.

In [37]:
print " ".join([token.text for token in ppatt.tokens])
for predicate in ppatt.instances:
    print "\t%s [%s-%s-%d]" % (" ".join(token.text for token in predicate.tokens),
                            predicate.root.text, predicate.root.gov_rel, predicate.root.position)
    for argument in predicate.arguments:
        print "\t\t%s [%s-%s-%d]" % (" ".join(token.text for token in argument.tokens),
                                     argument.root.text, argument.root.gov_rel, argument.root.position)

You wonder if he was manipulating the market with his bombing targets .
	wonder [wonder-root-1]
		You [You-nsubj-0]
	was manipulating with [manipulating-advcl-5]
		he [he-nsubj-3]
		the market [market-dobj-7]
		his bombing targets [targets-nmod-11]
	his [his-nmod:poss-9]
		his [his-nmod:poss-9]
		bombing targets [targets-nmod-11]


### Configurable options
PredPatt can be configured to tackle various syntactic phenomena. To do so, you can pass the configurable options to PredPatt.

In [38]:
from predpatt import PredPattOpts
resolve_relcl=True  # relative clauses
resolve_appos=True  # appositional modifiers
resolve_amod=True   # adjectival modifiers
resolve_conj=True   # conjuction
resolve_poss=True   # possessives
ud="v1"             # the version of UD (choices=["v1", "v2"])
opts = PredPattOpts(
        resolve_relcl=resolve_relcl,
        resolve_appos=resolve_appos,
        resolve_amod=resolve_amod,
        resolve_conj=resolve_conj,
        resolve_poss=resolve_poss,
        ud=ud
        )

ppatt = PredPatt(ud_example, opts=opts)
print " ".join([token.text for token in ppatt.tokens])
print ppatt.pprint(color=True, track_rule=False)

You wonder if he was manipulating the market with his bombing targets .
	?a wonder	[wonder-root]
		?a: You	[You-nsubj]
	?a was manipulating ?b with ?c	[manipulating-advcl]
		?a: he	[he-nsubj]
		?b: the market	[market-dobj]
		?c: his bombing targets	[targets-nmod]
	?a poss ?b	[his-nmod:poss]
		?a: his	[his-nmod:poss]
		?b: bombing targets	[targets-nmod]
